In [ ]:
!pip install indoxMiner paddlepaddle paddleocr 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import asyncio
import sys
from loguru import logger
from ocr_processor import OCRProcessor
from llms import AsyncIndoxApi, Extractor, Schema
from utils import Document

# Configure logging
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>",
    level="INFO"
)

# Define example usage of the Extractor for invoice data extraction.
# This script initializes the OCRProcessor to extract text from images,
# then configures the Extractor to use a predefined schema for invoices.

async def main():
    """
    Main asynchronous function to perform document extraction.

    - Defines image paths for OCR processing.
    - Uses the OCRProcessor to extract text from images.
    - Initializes the Indox API and sets up the Extractor with an invoice schema.
    - Processes extracted text and outputs results as a DataFrame.
    """

    # Define paths to images containing invoice data
    image_paths = ['/data/invoice_Aaron Bergman_36258-1.png', '/data/invoice_Aaron Bergman_36259-1.png']

    # Initialize OCR processor
    ocr_processor = OCRProcessor(model='paddle')

    # Process each image to extract text content
    documents = []
    for image_path in image_paths:
        # OCR extraction for each image
        text = ocr_processor.extract_text(image_path)
        documents.append(Document(page_content=text, metadata={"filename": image_path}))

    # Initialize the AsyncIndoxApi with an API key for LLM-based extraction
    llm = AsyncIndoxApi(api_key="INDOX_API_KEY")

    # Instantiate the Extractor with the schema for invoices
    # Schema.Invoice includes predefined fields like Invoice Number, Date, Customer Name, and Total Amount.
    extractor = Extractor(llm=llm, schema=Schema.Invoice)

    # Extract structured data from the documents using the invoice schema
    extraction_results = await extractor.extract_multiple(documents)

    # # Convert extraction results to a DataFrame for easy viewing and analysis
    # df = extractor.to_dataframe(extraction_results)
    # print("Extraction Results DataFrame:")
    # print(df)

    # Display extraction results with error checking
    if extraction_results.is_valid:
        # Iterate through valid results and display extracted data
        for result in extraction_results.get_valid_results():
            print("Extracted Data:", result)
    else:
        # If there are validation errors, display them
        print("Extraction errors occurred:", extraction_results.validation_errors)


# Run the main async function
if __name__ == "__main__":
    await main()


[2024/10/31 11:32:22] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c